In [1]:
import pandas as pd

## Data Acquisition

In [2]:
Collisions = pd.read_csv("NYC_Collisions.csv")

In [3]:
Weather = pd.read_csv("NYC_Weather_2016_2022.csv")

## Data Cleaning and Handling Inconsistencies